In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import annotations

import numpy as np

from mhsxtrapy.b3d import WhichSolution, b3d
from mhsxtrapy.field2d import Field2dData, FluxBalanceState
from mhsxtrapy.phibar import (dphidz, dphidz_low, dphidz_nw, phi, phi_low,
                              phi_nw)
from mhsxtrapy.plotting.vis import plot_magnetogram_2D

In [ ]:
"""
INSTANTIATE BOUNDARY CONDITION

We instantiate the boundary condition manually from the analytical expression given in the following cell using the function dipole(x, y).

We choose a grid resolution of 200 in x- and y-direction (and resulting number of Fourier modes = 200) as well as resolution in z-direction of 400.
We want the boundary condition to extend to 20L (= 20 Mm) in all directions. Therefore, the extrapolation will lead to a 3D data cube.
"""

nx, ny, nz, nf = 200, 200, 400, 200
xmin, xmax, ymin, ymax, zmin, zmax = 0.0, 20.0, 0.0, 20.0, 0.0, 20.0

"""
Calculation of pixel sizes and arrays of x-, y- and z-extension of box. 
"""
pixelsize_x = (xmax - xmin) / nx
pixelsize_y = (ymax - ymin) / ny
pixelsize_z = (zmax - zmin) / nz

x_arr = np.linspace(xmin, xmax, nx, dtype=np.float64)
y_arr = np.linspace(ymin, ymax, ny, dtype=np.float64)
z_arr = np.linspace(zmin, zmax, nz, dtype=np.float64)

In [ ]:
def dipole(x: np.float64, y: np.float64) -> np.float64:
    """
    Returns value of Dipole-Von Mises distribution at given x and y inspired by Neukirch and Wiegelmann (2019).
    mu_x1, mu_y1, mu_x2, mu_y2 as well as kappa_x1, kappa_y1, kappa_x2 and kappa_y2 can be changed to adjust positions and radii of sink and source.
    This is a simple example to test the runtime of the code.
    """

    xx = np.pi * (x / 10.0 - 1.0)
    yy = np.pi * (y / 10.0 - 1.0)
    mu_x1 = 1.2 / np.pi + 1.0
    mu_y1 = mu_x1
    mu_x2 = -mu_x1
    mu_y2 = -mu_y1
    kappa_x1 = 10.0
    kappa_y1 = kappa_x1
    kappa_x2 = kappa_y1
    kappa_y2 = kappa_x1

    return np.exp(kappa_x1 * np.cos(xx - mu_x1)) / (
        2.0 * np.pi * np.i0(kappa_x1)
    ) * np.exp(kappa_y1 * np.cos(yy - mu_y1)) / (
        2.0 * np.pi * np.i0(kappa_y1)
    ) - np.exp(
        kappa_x2 * np.cos(xx - mu_x2)
    ) / (
        2.0 * np.pi * np.i0(kappa_x2)
    ) * np.exp(
        kappa_y2 * np.cos(yy - mu_y2)
    ) / (
        2.0 * np.pi * np.i0(kappa_y2)
    )

In [ ]:
"""
INSTANTIATE BOUNDARY CONDITION 2

We define the normalising magnetic field strength as 500 Gauss. 
"""

B_PHOTO = 500

data_bz = np.zeros((ny, nx))

for ix in range(0, nx):
    for iy in range(0, ny):
        x = x_arr[ix] 
        y = y_arr[iy] 
        data_bz[iy, ix] = dipole(x, y)

In [ ]:
"""
INSTANTIATE Field2dData OBJECT FROM ANALYTICAL BOUNDARY CONDITION LABELLED AS "UNBALANCED" TO TEST RUNTIME WHEN Seehafer (1978) IS APPLIED
"""

data2d = Field2dData(
    nx,
    ny,
    nz,
    nf,
    pixelsize_x,
    pixelsize_y,
    pixelsize_z,
    x_arr,
    y_arr,
    z_arr,
    data_bz,
    flux_balance_state=FluxBalanceState.UNBALANCED,
)

In [ ]:
"""
PLOT 2D BOUNDARY CONDITION
"""

plot_magnetogram_2D(data2d)

In [ ]:
"""
DEFINE PARAMETERS FOR EXTRAPOLATION
"""

a = 0.22
alpha = 0.05
b = 1.0
z0 = 2.0
deltaz = 0.2

In [ ]:
"""
CALCULATE WAVE NUMBERS AND COEFFICIENTS FOR DIRECT CALCULATION OF \bar{\Phi} AND ITS FIRST DERIVATIVE WRT z

For details on the definition of \bar{\Phi} and its first derivative as well as the role they play in our model see the publications by

Neukirch, T., Wiegelmann, T. (2019)
Analytical Three-dimensional MagnetohydrostaticEquilibrium Solutions for Magnetic Field ExtrapolationAllowing a Transition from Non-force-free to Force-freeMagnetic Fields, 
Solar Physics,
https://doi.org/10.1007/s11207-019-1561-0

Nadol, L., Neukirch, T. (2025).
An efficient method for magnetic field extrapolation based on a family of analytical three-dimensional magnetohydrostatic equilibria,
Solar Physics,
https://doi.org/etc/etc/etc

Nadol, L., Neukirch, T. (2025).
MHSXtraPy - A set of codes for three-dimensional magnetohydrostatic solar magnetic field extrapolation,
RAS Techniques and Instruments,
https://doi.org/etc/etc/etc

"""

length_scale = float(2.0)  # Normalising length scale for Seehafer

length_scale_x = 2.0 * data2d.nx * data2d.px
length_scale_y = 2.0 * data2d.ny * data2d.py

length_scale_x_norm = length_scale_x / length_scale
length_scale_y_norm = length_scale_y / length_scale

kx_arr = np.arange(data2d.nf) * np.pi / length_scale_x_norm  # [0:nf_max]
ky_arr = np.arange(data2d.nf) * np.pi / length_scale_y_norm  # [0:nf_max]
one_arr = 0.0 * np.arange(data2d.nf) + 1.0

ky_grid = np.outer(ky_arr, one_arr)  # [0:nf_max, 0:nf_max]
kx_grid = np.outer(one_arr, kx_arr)  # [0:nf_max, 0:nf_max]

# kx^2 + ky^2

k2_arr = np.outer(ky_arr**2, one_arr) + np.outer(one_arr, kx_arr**2)
k2_arr[0, 0] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2
k2_arr[1, 0] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2
k2_arr[0, 1] = (np.pi / length_scale_x_norm) ** 2 + (np.pi / length_scale_y_norm) ** 2

p_arr = (
    0.5 * deltaz * np.sqrt(k2_arr[0:data2d.nf, 0:data2d.nf] * (1.0 - a - a * b) - alpha**2)
)
q_arr = (
    0.5 * deltaz * np.sqrt(k2_arr[0:data2d.nf, 0:data2d.nf] * (1.0 - a + a * b) - alpha**2)
)

In [ ]:
"""
INSTANTIATE ARRAYS FOR RUNTIME TESTS
"""

phi_asymp = np.zeros((data2d.nf, data2d.nf, data2d.nz))
dphidz_asymp = np.zeros_like(phi_asymp)
phi_NW = np.zeros_like(phi_asymp)
dphidz_NW = np.zeros_like(phi_asymp)

In [ ]:
"""
DEFINE FUNCTIONS FOR RUNTIME TESTING OF \bar{\Phi} AND ITS FIRST DERIVATIVE
"""

def check_runtime_phi_nn():
    for iz, z in enumerate(z_arr):
        phi_asymp[:, :, iz] = phi(z, p_arr, q_arr, z0, deltaz)

def check_runtime_phi_nw():
    for iz, z in enumerate(z_arr):
        phi_NW[:, :, iz] = phi_nw(z, p_arr, q_arr, z0, deltaz)

def check_runtime_dphidz_nn():
    for iz, z in enumerate(z_arr):
        dphidz_asymp[:, :, iz] = dphidz(z, p_arr, q_arr, z0, deltaz)

def check_runtime_dphidz_nw():
    for iz, z in enumerate(z_arr):
        dphidz_NW[:, :, iz] = dphidz_nw(z, p_arr, q_arr, z0, deltaz)

In [ ]:
"""
TEST RUNTIME OF \bar{Phi} FOR NEW ASYMPTOTIC SOLUTION 
"""

%timeit -n 100 -r 10 check_runtime_phi_nn()

In [ ]:
"""
TEST RUNTIME OF \bar{Phi} FOR Neukirch and Wiegelmann (2019) SOLUTION 
"""

%timeit -n 100 -r 10 check_runtime_phi_nw()

In [ ]:
"""
TEST RUNTIME OF FIRST DERIVATIVE OF \bar{Phi} FOR NEW ASYMPTOTIC SOLUTION 
"""

%timeit -n 100 -r 10 check_runtime_dphidz_nn()

In [ ]:
"""
TEST RUNTIME OF FIRST DERIVATIVE OF \bar{Phi} FOR Neukirch adn Wiegelmann (2019) SOLUTION 
"""

%timeit -n 100 -r 10 check_runtime_dphidz_nw()

In [ ]:
"""
DEFINE FUNCTIONS FOR RUNTIME TESTING OF B CALUCULATION IN UNBALANCED CASE FOR BOTH THE ASYMPTOTIC AND THE Neukirch and Wiegelmann (2019) SOLUTIONS
"""

def check_runtime_b_nn_unbalanced():
    b3d(data2d, a=a, b=b, alpha=alpha, z0=z0, deltaz=deltaz, solution=WhichSolution.ASYMP)

def check_runtime_b_nw_unbalanced():
    b3d(data2d, a=a, b=b, alpha=alpha, z0=z0, deltaz=deltaz, solution=WhichSolution.NEUWIE)

In [ ]:
"""
INSTANTIATE Field2dData OBJECT FROM ANALYTICAL BOUNDARY CONDITION LABELLED AS "BALANCED" TO ALSO TEST RUNTIME WHEN Seehafer (1978) IS NOT APPLIED
"""

data2d_balanced = Field2dData(
    nx,
    ny,
    nz,
    nf,
    pixelsize_x,
    pixelsize_y,
    pixelsize_z,
    x_arr,
    y_arr,
    z_arr,
    data_bz,
    flux_balance_state=FluxBalanceState.BALANCED,
)

In [ ]:
"""
DEFINE FUNCTIONS FOR RUNTIME TESTING OF B CALUCULATION IN BALANCED CASE FOR BOTH THE ASYMPTOTIC AND THE Neukirch and Wiegelmann (2019) SOLUTIONS
"""

def check_runtime_b_nn_balanced():
    b3d(data2d_balanced, a=a, b=b, alpha=alpha, z0=z0, deltaz=deltaz, solution=WhichSolution.ASYMP)

def check_runtime_b_nw_balanced():
    b3d(data2d_balanced, a=a, b=b, alpha=alpha, z0=z0, deltaz=deltaz, solution=WhichSolution.NEUWIE)

In [ ]:
"""
TEST RUNTIME OF b3d IN UNBALANCED CASE WITH ASYMPTOTIC SOLUTION
"""

%timeit -n 5 -r 5 check_runtime_b_nn_unbalanced()

In [ ]:
"""
TEST RUNTIME OF b3d IN UNBALANCED CASE WITH Neukirch and Wiegelmann (2019) SOLUTION
"""

%timeit -n 5 -r 5 check_runtime_b_nw_unbalanced()

In [ ]:
"""
TEST RUNTIME OF b3d IN BALANCED CASE WITH ASYMPTOTIC SOLUTION
"""

%timeit -n 5 -r 5 check_runtime_b_nn_balanced()

In [ ]:
"""
TEST RUNTIME OF b3d IN BALANCED CASE WITH Neukirch and Wiegelmann (2019) SOLUTION
"""

%timeit -n 5 -r 5 check_runtime_b_nw_balanced()